In [35]:
import time
import requests
import json

def exponential_backoff_retry(
    func, 
    title,
    max_retries=5, 
    base_delay=1, 
    max_delay=32
):
    """
    Retries a function with exponential backoff.

    Args:
        func: A callable that may raise an exception.
        max_retries: Maximum number of retries before giving up.
        base_delay: Initial delay in seconds.
        max_delay: Maximum delay in seconds.
        jitter: Whether to add randomness to the delay.

    Returns:
        The result of the function if successful.

    Raises:
        The last exception raised if all retries fail.
    """
    for attempt in range(1, max_retries + 1):
        try:
            return func(title)
        except Exception as e:
            if attempt == max_retries:
                raise  # Re-raise the last exception
            else:
                delay = min(base_delay * 2**(attempt - 1), max_delay)
                print(f"Attempt {attempt} failed. Retrying in {delay:.2f} seconds...")
                time.sleep(delay)

def search_paper_by_title(title):
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": f"title:({title})",
        "fields": "title,url,publicationTypes,publicationDate,openAccessPdf,citationCount,authors,abstract",
        "year": "2020-",
        "limit": 1  # Adjust as needed
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if data.get("data"):
            return data["data"][0]  # Return the first matching paper
        else:
            print("No matching papers found.")
    else:
        print(f"Error: {response.status_code}")
    return None

In [36]:
paper_titles = [
    'Latent Space Editing in Transformer-Based Flow Matching', 
    'A Survey of Multimodal Controllable Diffusion Models'
]

data = []
for title in paper_titles:
    paper_data = exponential_backoff_retry(search_paper_by_title, title)
    data.append(paper_data)


No matching papers found.


In [37]:
data

[{'paperId': 'ca743e75ce090bbf686307e41bd8747661768fbe',
  'url': 'https://www.semanticscholar.org/paper/ca743e75ce090bbf686307e41bd8747661768fbe',
  'title': 'Latent Space Editing in Transformer-Based Flow Matching',
  'abstract': 'This paper strives for image editing via generative models. Flow Matching is an emerging generative modeling technique that offers the advantage of simple and efficient training. Simultaneously, a new transformer-based U-ViT has recently been proposed to replace the commonly used UNet for better scalability and performance in generative modeling. Hence, Flow Matching with a transformer backbone offers the potential for scalable and high-quality generative modeling, but their latent structure and editing ability are as of yet unknown. Hence, we adopt this setting and explore how to edit images through latent space manipulation. We introduce an editing space, which we call u-space, that can be manipulated in a controllable, accumulative, and composable manner